In [ ]:
# Import required modules
from langchain_ollama import ChatOllama
from langchain_groq  import ChatGroq
from langchain.agents import create_agent, AgentState
from langgraph.runtime import Runtime
import custom_tools

In [32]:
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

basic_model = ChatOllama(model="mistral:7b",base_url="http://localhost:11434",temperature=0.0)
advanced_model = ChatGroq(model="openai/gpt-oss-20b", temperature=0.0)

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 1:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    return handler(request.override(model=model))



In [34]:
# Create agent with dynamic model selection
tool_list = [custom_tools.search_internet, custom_tools.calculate]
agent = create_agent(
    model=basic_model,  # Default model
    tools=tool_list,
    middleware=[dynamic_model_selection]
)

print("Dynamic agent created successfully!")
print("This agent will automatically switch models based on conversation complexity")

Dynamic agent created successfully!
This agent will automatically switch models based on conversation complexity


In [45]:
result1 = agent.invoke({
    "messages": ["Difference between Agentic AI and Generative AI in simple terms?", "Search latest news in India", "What is the capital of France?"]
})

print(result1)

{'messages': [HumanMessage(content='Difference between Agentic AI and Generative AI in simple terms?', additional_kwargs={}, response_metadata={}, id='50b36094-f640-41bb-b483-4f5811f4b10b'), HumanMessage(content='Search latest news in India', additional_kwargs={}, response_metadata={}, id='6c81b3b3-6aca-4792-8b90-95f337da0fc0'), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}, id='40062495-5cad-4547-9336-9f4e7171f97b'), AIMessage(content='', additional_kwargs={'reasoning_content': 'We have multiple user messages: first "Difference between Agentic AI and Generative AI in simple terms?" then "Search latest news in India" then "What is the capital of France?" The user is asking multiple questions. We need to answer all. The second question requires a search. We can use the search_internet function. The third is trivial: capital of France is Paris. The first is explanation. So we need to produce a single response covering all three. We shou

In [46]:
print(result1["messages"][-1].content)

**1. Agentic AI vs. Generative AI (in plain terms)**  

| Feature | Agentic AI | Generative AI |
|---------|------------|---------------|
| **Primary goal** | To act *on behalf of a user* – it plans, decides, and takes actions to achieve a specific objective (e.g., booking a flight, managing a portfolio). | To *create* new content – it produces text, images, music, code, etc., based on patterns it learned from data. |
| **How it works** | Uses a combination of planning, decision‑making, and sometimes reinforcement learning to choose the best next step. It often interacts with external tools or APIs. | Uses large language or multimodal models that generate outputs token‑by‑token (or pixel‑by‑pixel) from a probability distribution. |
| **Typical use‑cases** | Virtual assistants, autonomous robots, recommendation engines that need to follow a strategy. | Chatbots, creative writing, image synthesis, code generation, data augmentation. |
| **Key difference** | *Action‑oriented* – it’s about